# Analyze Upper Bound of Type I Error for Lei Example

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import jax
import os
import numpy as np
from confirm.mini_imprint import grid
from confirm.lewislib import grid as lewgrid
from confirm.lewislib import lewis
from confirm.berrylib import binomial

ModuleNotFoundError: No module named 'outlaw.interp'

In [4]:
# Configuration used during simulation
params = {
    "n_arms" : 4,
    "n_stage_1" : 50,
    "n_stage_2" : 100,
    "n_stage_1_interims" : 2,
    "n_stage_1_add_per_interim" : 100,
    "n_stage_2_add_per_interim" : 100,
    "stage_1_futility_threshold" : 0.15,
    "stage_1_efficacy_threshold" : 0.7,
    "stage_2_futility_threshold" : 0.2,
    "stage_2_efficacy_threshold" : 0.95,
    "inter_stage_futility_threshold" : 0.6,
    "posterior_difference_threshold" : 0,
    "rejection_threshold" : 0.05,
    "key" : jax.random.PRNGKey(0),
    "n_pr_sims" : 100,
    "n_sig2_sims" : 20,
    "batch_size" : int(2**20),
    "cache_tables" : False,
}
size = 52
n_sim_batches = 500
sim_batch_size = 100

In [5]:
# construct Lei object
lei_obj = lewis.Lewis45(**params)

In [6]:
# construct the same grid used during simulation
n_arms = params['n_arms']
lower = np.full(n_arms, -1)
upper = np.full(n_arms, 1)
thetas, radii = lewgrid.make_cartesian_grid_range(
    size=size,
    lower=lower,
    upper=upper,
)
ns = np.concatenate(
    [np.ones(n_arms-1)[:, None], -np.eye(n_arms-1)],
    axis=-1,
)
null_hypos = [
    grid.HyperPlane(n, 0)
    for n in ns
]
gr = grid.build_grid(
    thetas=thetas,
    radii=radii,
    null_hypos=null_hypos,
)
gr = grid.prune(gr)

In [7]:
# construct tile informations used during simulation
theta_tiles = gr.thetas[gr.grid_pt_idx]
p_tiles = jax.scipy.special.expit(theta_tiles)
tile_radii = gr.radii[gr.grid_pt_idx]
null_truths = gr.null_truth.astype(bool)
sim_size = 2 * n_sim_batches * sim_batch_size  # 2 instances parallelized
sim_sizes = np.full(gr.n_tiles, sim_size)

In [12]:
# get type I sum and score
cwd = '.'
data_dir = os.path.join(cwd, '../data')
output_dir = os.path.join(data_dir, 'output_1')
typeI_sum = np.loadtxt(os.path.join(output_dir, 'typeI_sum.csv'), delimiter=',')
typeI_score = np.loadtxt(os.path.join(output_dir, 'typeI_score.csv'), delimiter=',')
output_dir = os.path.join(data_dir, 'output_2')
typeI_sum += np.loadtxt(os.path.join(output_dir, 'typeI_sum.csv'), delimiter=',')
typeI_score += np.loadtxt(os.path.join(output_dir, 'typeI_score.csv'), delimiter=',')

In [15]:
typeI_sum / sim_size, typeI_score / sim_size

(array([1.12e-03, 6.70e-04, 6.40e-04, ..., 4.00e-05, 4.30e-04, 5.20e-04]),
 array([[-1.71344219e-02,  7.10989403e-03,  5.46108647e-03,
          4.81012845e-03],
        [-1.03829813e-02,  5.86771274e-03,  5.34518179e-03,
         -1.27500590e-03],
        [-1.02029159e-02,  5.75778215e-03,  5.16429055e-03,
         -1.30454266e-03],
        ...,
        [-5.08045978e-04,  7.36710084e-05, -7.63368492e-05,
          9.59055195e-04],
        [-7.03037203e-03,  2.41161544e-03,  3.35358649e-03,
          1.13478709e-03],
        [-8.25278119e-03,  2.67263239e-03,  2.21990762e-03,
          3.11372006e-03]]))

In [16]:
# construct upper bound
n_arm_samples = lei_obj.unifs_shape()[0]
total, d0, d0u, d1w, d1uw, d2uw = binomial.upper_bound(
    theta_tiles,
    tile_radii,
    gr.vertices,
    sim_sizes,
    n_arm_samples,
    typeI_sum,
    typeI_score,
)
bound_components = np.array([
    d0,
    d0u,
    d1w,
    d1uw,
    d2uw,
    total,
]).T

In [17]:
t2_uniques = np.unique(theta_tiles[:, 2])
t3_uniques = np.unique(theta_tiles[:, 3])
t2_uniques, t3_uniques

(array([-0.98076923, -0.94230769, -0.90384615, -0.86538462, -0.82692308,
        -0.78846154, -0.75      , -0.71153846, -0.67307692, -0.63461538,
        -0.59615385, -0.55769231, -0.51923077, -0.48076923, -0.44230769,
        -0.40384615, -0.36538462, -0.32692308, -0.28846154, -0.25      ,
        -0.21153846, -0.17307692, -0.13461538, -0.09615385, -0.05769231,
        -0.01923077,  0.01923077,  0.05769231,  0.09615385,  0.13461538,
         0.17307692,  0.21153846,  0.25      ,  0.28846154,  0.32692308,
         0.36538462,  0.40384615,  0.44230769,  0.48076923,  0.51923077,
         0.55769231,  0.59615385,  0.63461538,  0.67307692,  0.71153846,
         0.75      ,  0.78846154,  0.82692308,  0.86538462,  0.90384615,
         0.94230769,  0.98076923]),
 array([-0.98076923, -0.94230769, -0.90384615, -0.86538462, -0.82692308,
        -0.78846154, -0.75      , -0.71153846, -0.67307692, -0.63461538,
        -0.59615385, -0.55769231, -0.51923077, -0.48076923, -0.44230769,
        -0.4038

In [24]:
# slice and save P, B
t2 = t2_uniques[25]
t3 = t3_uniques[20]
selection = (theta_tiles[:, 2] == t2) & (theta_tiles[:, 3] == t3)

In [25]:
bound_dir = os.path.join(data_dir, 'bound')
if not os.path.exists(bound_dir):
    os.makedirs(bound_dir)

np.savetxt(f'{bound_dir}/P_lei.csv', theta_tiles[selection, :].T, fmt="%s", delimiter=",")
np.savetxt(f'{bound_dir}/B_lei.csv', bound_components[selection, :], fmt="%s", delimiter=",")